In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


# Load Libraries and dataset

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import optuna


In [3]:

train_data = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

# Feature Engineering and Preprocessing

***Convert categorical variables to numerical using Label Encoding and one hot encoding***

In [4]:
gender_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
gender_encoded = gender_encoder.fit_transform(train_data[['Gender']])
train_data[['Gender_Female', 'Gender_Male']] = gender_encoded
test_data[['Gender_Female', 'Gender_Male']] = gender_encoder.transform(test_data[['Gender']])

geography_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
geography_encoded = geography_encoder.fit_transform(train_data[['Geography']])
train_data[['Geography_France', 'Geography_Spain', 'Geography_Germany']] = geography_encoded
test_data[['Geography_France', 'Geography_Spain', 'Geography_Germany']] = geography_encoder.transform(test_data[['Geography']])

surname_frequency = train_data['Surname'].value_counts(normalize=True)
train_data['Surname_Frequency'] = train_data['Surname'].map(surname_frequency)
test_data['Surname_Frequency'] = test_data['Surname'].map(surname_frequency)


train_data.drop(['Gender', 'Geography', 'Surname'], axis=1, inplace=True)
test_data.drop(['Gender', 'Geography', 'Surname'], axis=1, inplace=True)


**Split data into features and target**

In [5]:
X = train_data.drop('Exited', axis=1)
y = train_data['Exited']

**Data Scaling**

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

***Train-Test Split***

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Model Building and training

***Hyperparameter Tuning using Optuna for XGBoost***

In [8]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
    }
    xgb_model = XGBClassifier(**params)
    xgb_model.fit(X_train, y_train)
    y_val_pred_proba = xgb_model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred_proba)
    return roc_auc

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=100)

best_xgb_params = study_xgb.best_params
best_xgb = XGBClassifier(**best_xgb_params)
best_xgb.fit(X_train, y_train)

[I 2024-01-31 11:21:08,141] A new study created in memory with name: no-name-0f43d030-11f8-4c4c-a47c-ba71550784cb
[I 2024-01-31 11:21:10,451] Trial 0 finished with value: 0.8938844674197478 and parameters: {'n_estimators': 199, 'learning_rate': 0.10475519622594523, 'max_depth': 6}. Best is trial 0 with value: 0.8938844674197478.
[I 2024-01-31 11:21:12,506] Trial 1 finished with value: 0.8937470797496972 and parameters: {'n_estimators': 174, 'learning_rate': 0.09301098385219779, 'max_depth': 6}. Best is trial 0 with value: 0.8938844674197478.
[I 2024-01-31 11:21:13,333] Trial 2 finished with value: 0.8930816131382646 and parameters: {'n_estimators': 50, 'learning_rate': 0.1822282814289587, 'max_depth': 7}. Best is trial 0 with value: 0.8938844674197478.
[I 2024-01-31 11:21:14,535] Trial 3 finished with value: 0.8937869380963781 and parameters: {'n_estimators': 114, 'learning_rate': 0.1721720083704531, 'max_depth': 5}. Best is trial 0 with value: 0.8938844674197478.
[I 2024-01-31 11:21:1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1311417583541052,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=176, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

**Hyperparameter Tuning using Optuna for LightGBM**

In [9]:
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
    }
    lgbm_model = LGBMClassifier(**params)
    lgbm_model.fit(X_train, y_train)
    y_val_pred_proba = lgbm_model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred_proba)
    return roc_auc

study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(objective_lgbm, n_trials=100)

best_lgbm_params = study_lgbm.best_params
best_lgbm = LGBMClassifier(**best_lgbm_params)
best_lgbm.fit(X_train, y_train)

[I 2024-01-31 11:24:14,446] A new study created in memory with name: no-name-70f07eb5-3e19-432f-8ed0-9166ea8d8be4


[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-01-31 11:24:16,118] Trial 0 finished with value: 0.8929518306747674 and parameters: {'n_estimators': 112, 'learning_rate': 0.16454645926804187, 'max_depth': 3}. Best is trial 0 with value: 0.8929518306747674.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-01-31 11:24:20,588] Trial 1 finished with value: 0.8918431841730463 and parameters: {'n_estimators': 181, 'learning_rate': 0.06721158037026437, 'max_depth': 3}. Best is trial 0 with value: 0.8929518306747674.


...

[I 2024-01-31 11:31:52,566] Trial 98 finished with value: 0.8942195187129474 and parameters: {'n_estimators': 190, 'learning_rate': 0.1698031755421143, 'max_depth': 4}. Best is trial 77 with value: 0.8946982162424032.


[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2024-01-31 11:31:57,188] Trial 99 finished with value: 0.8940604495813992 and parameters: {'n_estimators': 183, 'learning_rate': 0.18923722937314727, 'max_depth': 4}. Best is trial 77 with value: 0.8946982162424032.


[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

LGBMClassifier(learning_rate=0.1980808446142959, max_depth=4, n_estimators=199)

**Hyperparameter Tuning using Optuna for CatBoost**

In [10]:
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'thread_count': 4,
        'verbose': 0,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
    }
    catboost_model = CatBoostClassifier(**params)
    catboost_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=100)
    y_val_pred_proba = catboost_model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred_proba)
    return roc_auc

study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(objective_catboost, n_trials=100)

best_catboost_params = study_catboost.best_params
best_catboost = CatBoostClassifier(**best_catboost_params)
best_catboost.fit(X_train, y_train)

[I 2024-01-31 11:32:02,198] A new study created in memory with name: no-name-15d51077-6fa4-415e-a23f-d99641be164f
/tmp/ipykernel_18/546627863.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),


0:	test: 0.8807178	best: 0.8807178 (0)	total: 83.3ms	remaining: 6.49s
78:	test: 0.8921694	best: 0.8921853 (64)	total: 1.82s	remaining: 0us

bestTest = 0.8921853274
bestIteration = 64

Shrink model to first 65 iterations.


[I 2024-01-31 11:32:05,381] Trial 0 finished with value: 0.8921853274041421 and parameters: {'iterations': 79, 'learning_rate': 0.1836006221170636, 'depth': 9, 'l2_leaf_reg': 0.47335610865490835, 'border_count': 79}. Best is trial 0 with value: 0.8921853274041421.
/tmp/ipykernel_18/546627863.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),


0:	test: 0.8514349	best: 0.8514349 (0)	total: 13.8ms	remaining: 1.03s
75:	test: 0.8823712	best: 0.8823712 (75)	total: 1s	remaining: 0us

bestTest = 0.8823712112
bestIteration = 75



[I 2024-01-31 11:32:07,594] Trial 1 finished with value: 0.8823712112356606 and parameters: {'iterations': 76, 'learning_rate': 0.029648625288121032, 'depth': 4, 'l2_leaf_reg': 0.14506336572805692, 'border_count': 93}. Best is trial 0 with value: 0.8921853274041421.
/tmp/ipykernel_18/546627863.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),


...

0:	test: 0.8802503	best: 0.8802503 (0)	total: 20.2ms	remaining: 3.87s
100:	test: 0.8931080	best: 0.8931356 (95)	total: 1.81s	remaining: 1.65s
192:	test: 0.8931605	best: 0.8934361 (150)	total: 3.32s	remaining: 0us

bestTest = 0.8934360721
bestIteration = 150

Shrink model to first 151 iterations.


[I 2024-01-31 11:38:35,280] Trial 98 finished with value: 0.8934360720576212 and parameters: {'iterations': 193, 'learning_rate': 0.16873401357382425, 'depth': 7, 'l2_leaf_reg': 0.9170127743069535, 'border_count': 198}. Best is trial 95 with value: 0.8943313947231348.
/tmp/ipykernel_18/546627863.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),


0:	test: 0.8706581	best: 0.8706581 (0)	total: 17.3ms	remaining: 3.24s
100:	test: 0.8935304	best: 0.8935304 (100)	total: 1.57s	remaining: 1.35s
187:	test: 0.8938228	best: 0.8938360 (180)	total: 2.85s	remaining: 0us

bestTest = 0.8938360463
bestIteration = 180

Shrink model to first 181 iterations.


[I 2024-01-31 11:38:39,365] Trial 99 finished with value: 0.893836046316922 and parameters: {'iterations': 188, 'learning_rate': 0.18084410843122495, 'depth': 6, 'l2_leaf_reg': 1.8318034139009993, 'border_count': 189}. Best is trial 95 with value: 0.8943313947231348.


0:	learn: 0.5995846	total: 12.9ms	remaining: 2.54s
1:	learn: 0.5341513	total: 28ms	remaining: 2.73s
2:	learn: 0.4851726	total: 43.5ms	remaining: 2.81s
3:	learn: 0.4504070	total: 58.4ms	remaining: 2.82s
4:	learn: 0.4221885	total: 72.6ms	remaining: 2.79s
5:	learn: 0.4011464	total: 87.3ms	remaining: 2.78s
6:	learn: 0.3843153	total: 103ms	remaining: 2.8s
7:	learn: 0.3720261	total: 117ms	remaining: 2.77s
8:	learn: 0.3618844	total: 130ms	remaining: 2.72s
9:	learn: 0.3534818	total: 144ms	remaining: 2.7s
10:	learn: 0.3477520	total: 159ms	remaining: 2.69s
11:	learn: 0.3423068	total: 173ms	remaining: 2.66s
12:	learn: 0.3383669	total: 186ms	remaining: 2.64s
13:	learn: 0.3358739	total: 198ms	remaining: 2.59s
14:	learn: 0.3330475	total: 210ms	remaining: 2.55s
15:	learn: 0.3305255	total: 227ms	remaining: 2.56s
16:	learn: 0.3289241	total: 241ms	remaining: 2.56s
17:	learn: 0.3278495	total: 255ms	remaining: 2.54s
18:	learn: 0.3262334	total: 270ms	remaining: 2.53s
19:	learn: 0.3252511	total: 284ms	remai

**Ensemble using Voting Classifier**

In [11]:
ensemble_model = VotingClassifier(estimators=[
    ('xgb', best_xgb),
    ('lgbm', best_lgbm),
    ('catboost', best_catboost)
], voting='soft')


**Training the ensemble model**

In [12]:
ensemble_model.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_ra...
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=176, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.1980808446142959,
                                             max_depth=4, n_estimators=199)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x79d98ad7a260>)],
                 voting='soft')

**Create and fit imputer**

In [13]:
imputer = SimpleImputer(strategy='mean')  
X_scaled_imputed = imputer.fit_transform(X_scaled)

# Model Evaluation

**Use validation set for evaluation**

In [14]:
y_val_pred_proba = ensemble_model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred_proba)
print(f'ROC AUC Score on Validation Set: {roc_auc}')


ROC AUC Score on Validation Set: 0.8954170959082774


# Make Predictions on Test Data

In [15]:
X_test_scaled = scaler.transform(test_data)
X_test_imputed = imputer.transform(X_test_scaled)
test_predictions = ensemble_model.predict_proba(X_test_imputed)[:, 1]

# Create Submission File

In [16]:
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_predictions})
submission_df.to_csv('submission.csv', index=False)